In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from utils import *

In [2]:
root = "../../data/tencent2020/"
path_buld = root + "build/"
path_buld2 = root + "build2/"
path_save = root + "save/"
train_preliminary_p = path_buld + "train_preliminary/"
train_preliminary_p2 = path_buld2 + "train_semi_final/"
test_p = path_buld + "test/"

In [3]:
train_ad = pd.read_csv(train_preliminary_p + "ad.csv", encoding="utf-8")
train_click_log = pd.read_csv(train_preliminary_p + "click_log.csv", encoding="utf-8")
train_user = pd.read_csv(train_preliminary_p + "user.csv", encoding="utf-8")

train_ad2 = pd.read_csv(train_preliminary_p2 + "ad.csv", encoding="utf-8")
train_click_log2 = pd.read_csv(train_preliminary_p2 + "click_log.csv", encoding="utf-8")
train_user2 = pd.read_csv(train_preliminary_p2 + "user.csv", encoding="utf-8")

test_ad = pd.read_csv(test_p + "ad.csv", encoding="utf-8")
test_click_log = pd.read_csv(test_p + "click_log.csv", encoding="utf-8")

In [4]:
train_ad_all = pd.concat([train_ad, train_ad2, test_ad], axis=0).drop_duplicates()   #(4445720, 6)
train_click_log_all = pd.concat([train_click_log, train_click_log2, test_click_log], axis=0)  #(133878445, 4)
train_user_all = pd.concat([train_user, train_user2], axis=0) # 3000000

In [10]:
train_ad_all['industry'] = train_ad_all['industry'].replace('\\N', '100').astype(int)
train_ad_all['product_id'] = train_ad_all['product_id'].replace('\\N', '10000').astype(int)
train_ad_all.loc[train_ad_all['product_id'] == 10000, 'product_id'] = 10000 + \
    train_ad_all.loc[train_ad_all['product_id'] == 10000, 'product_category']

In [12]:
click_log = train_click_log_all.merge(train_ad_all, on='creative_id', how='left')
data = click_log.merge(train_user_all, on='user_id', how='left') #(133878445, 11)

In [13]:
data['age'] = data['age'].fillna(0)
data['age'] = data['age'].astype(int)
data['age'] = data['age'] - 1

data['gender'] = data['gender'].fillna(0)
data['gender'] = data['gender'].astype(int)
data['gender'] = data['gender'] - 1

reduce_mem_usage(data) # Mem. usage decreased to 4851.70 Mb (58.7% reduction)
data.info()

Mem. usage decreased to 4596.35 Mb (57.1% reduction)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133878445 entries, 0 to 133878444
Data columns (total 11 columns):
time                int8
user_id             int32
creative_id         int32
click_times         int16
ad_id               int32
product_id          int32
product_category    int8
advertiser_id       int32
industry            int16
age                 int8
gender              int8
dtypes: int16(2), int32(5), int8(4)
memory usage: 4.5 GB


In [ ]:
data.to_csv(train_preliminary_p2 + "click_log_all.csv", encoding='utf-8', index=False)
